In [1]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/f2bcf/anaconda/envs/SNF-env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [5]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [15]:
questions[0:10]

['  85+99',
 '    1+3',
 '  449+5',
 '    9+8',
 ' 69+484',
 ' 22+026',
 '   5+36',
 '  8+119',
 ' 24+248',
 '    0+1']

In [16]:
expected[0:10]

['157 ',
 '4   ',
 '949 ',
 '17  ',
 '580 ',
 '642 ',
 '68  ',
 '919 ',
 '884 ',
 '1   ']

In [6]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [7]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [8]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

In [9]:
print('Training Data:')
print(x_train.shape)
print(y_train.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)


In [10]:
print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [11]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [12]:
print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

Build model...


In [17]:
# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 565us/step - loss: 1.8821 - acc: 0.3225 - val_loss: 1.7972 - val_acc: 0.3426
Q 739+518 T 1257 ☒ 1117
Q 135+988 T 1123 ☒ 111 
Q 981+537 T 1518 ☒ 1117
Q 343+2   T 345  ☒ 61  
Q 503+80  T 583  ☒ 161 
Q 644+40  T 684  ☒ 161 
Q 600+868 T 1468 ☒ 1117
Q 951+140 T 1091 ☒ 101 
Q 598+100 T 698  ☒ 111 
Q 23+30   T 53   ☒ 61  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 562us/step - loss: 1.7441 - acc: 0.3562 - val_loss: 1.6809 - val_acc: 0.3771
Q 230+7   T 237  ☒ 22  
Q 73+749  T 822  ☒ 800 
Q 490+624 T 1114 ☒ 100 
Q 632+54  T 686  ☒ 555 
Q 663+877 T 1540 ☒ 1310
Q 570+674 T 1244 ☒ 1355
Q 436+770 T 1206 ☒ 1310
Q 896+555 T 1451 ☒ 1110
Q 57+280  T 337  ☒ 788 
Q 286+553 T 839  ☒ 113 

-----------------

45000/45000 [==============================] - 25s 553us/step - loss: 0.3043 - acc: 0.9104 - val_loss: 0.2664 - val_acc: 0.9203
Q 879+3   T 882  ☑ 882 
Q 736+44  T 780  ☑ 780 
Q 422+2   T 424  ☑ 424 
Q 17+89   T 106  ☒ 107 
Q 37+49   T 86   ☒ 75  
Q 14+118  T 132  ☑ 132 
Q 492+218 T 710  ☑ 710 
Q 56+442  T 498  ☑ 498 
Q 95+930  T 1025 ☑ 1025
Q 822+92  T 914  ☑ 914 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 562us/step - loss: 0.2018 - acc: 0.9542 - val_loss: 0.1720 - val_acc: 0.9609
Q 495+451 T 946  ☑ 946 
Q 293+525 T 818  ☑ 818 
Q 89+105  T 194  ☑ 194 
Q 935+7   T 942  ☑ 942 
Q 450+2   T 452  ☑ 452 
Q 72+18   T 90   ☑ 90  
Q 201+5   T 206  ☑ 206 
Q 84+315  T 399  ☑ 399 
Q 637+2   T 639  ☑ 639 
Q 301+923 T 1224 ☑ 1224

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 25s 552us/step - loss: 0.0337 - acc: 0.9910 - val_loss: 0.1808 - val_acc: 0.9415
Q 59+61   T 120  ☑ 120 
Q 73+972  T 1045 ☑ 1045
Q 306+44  T 350  ☑ 350 
Q 736+661 T 1397 ☑ 1397
Q 309+6   T 315  ☑ 315 
Q 249+5   T 254  ☑ 254 
Q 614+41  T 655  ☑ 655 
Q 4+543   T 547  ☑ 547 
Q 883+23  T 906  ☑ 906 
Q 491+8   T 499  ☑ 499 

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 559us/step - loss: 0.0236 - acc: 0.9946 - val_loss: 0.0152 - val_acc: 0.9971
Q 89+105  T 194  ☑ 194 
Q 176+60  T 236  ☑ 236 
Q 670+295 T 965  ☑ 965 
Q 6+67    T 73   ☑ 73  
Q 706+444 T 1150 ☑ 1150
Q 412+851 T 1263 ☑ 1263
Q 53+991  T 1044 ☑ 1044
Q 577+5   T 582  ☑ 582 
Q 568+709 T 1277 ☑ 1277
Q 53+682  T 735  ☑ 735 

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 25s 552us/step - loss: 0.0031 - acc: 0.9998 - val_loss: 0.0055 - val_acc: 0.9991
Q 901+38  T 939  ☑ 939 
Q 170+68  T 238  ☑ 238 
Q 648+884 T 1532 ☑ 1532
Q 118+8   T 126  ☑ 126 
Q 498+981 T 1479 ☑ 1479
Q 370+33  T 403  ☑ 403 
Q 428+603 T 1031 ☑ 1031
Q 82+770  T 852  ☑ 852 
Q 69+754  T 823  ☑ 823 
Q 46+100  T 146  ☑ 146 

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 556us/step - loss: 0.0029 - acc: 0.9998 - val_loss: 0.0055 - val_acc: 0.9989
Q 930+468 T 1398 ☑ 1398
Q 768+252 T 1020 ☑ 1020
Q 850+658 T 1508 ☑ 1508
Q 148+4   T 152  ☑ 152 
Q 393+187 T 580  ☑ 580 
Q 92+259  T 351  ☑ 351 
Q 382+568 T 950  ☑ 950 
Q 244+118 T 362  ☑ 362 
Q 1+500   T 501  ☑ 501 
Q 117+916 T 1033 ☑ 1033

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 26s 583us/step - loss: 0.0397 - acc: 0.9883 - val_loss: 0.0336 - val_acc: 0.9892
Q 39+307  T 346  ☑ 346 
Q 346+75  T 421  ☑ 421 
Q 198+4   T 202  ☑ 202 
Q 94+865  T 959  ☑ 959 
Q 871+64  T 935  ☑ 935 
Q 72+104  T 176  ☑ 176 
Q 269+113 T 382  ☑ 382 
Q 835+577 T 1412 ☑ 1412
Q 24+526  T 550  ☑ 550 
Q 502+16  T 518  ☑ 518 

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 26s 576us/step - loss: 0.0067 - acc: 0.9985 - val_loss: 0.0047 - val_acc: 0.9990
Q 925+64  T 989  ☑ 989 
Q 549+1   T 550  ☑ 550 
Q 134+604 T 738  ☑ 738 
Q 665+531 T 1196 ☑ 1196
Q 18+35   T 53   ☑ 53  
Q 899+74  T 973  ☑ 973 
Q 192+89  T 281  ☑ 281 
Q 596+73  T 669  ☑ 669 
Q 104+2   T 106  ☑ 106 
Q 96+8    T 104  ☑ 104 

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

...

45000/45000 [==============================] - 25s 548us/step - loss: 2.2567e-04 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 0.9997
Q 247+749 T 996  ☑ 996 
Q 49+59   T 108  ☑ 108 
Q 810+895 T 1705 ☑ 1705
Q 17+436  T 453  ☑ 453 
Q 786+60  T 846  ☑ 846 
Q 0+384   T 384  ☑ 384 
Q 53+818  T 871  ☑ 871 
Q 503+429 T 932  ☑ 932 
Q 39+421  T 460  ☑ 460 
Q 75+497  T 572  ☑ 572 

--------------------------------------------------
Iteration 142
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 547us/step - loss: 2.0759e-04 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 0.9996
Q 40+99   T 139  ☑ 139 
Q 468+311 T 779  ☑ 779 
Q 29+66   T 95   ☑ 95  
Q 335+38  T 373  ☑ 373 
Q 36+77   T 113  ☑ 113 
Q 163+91  T 254  ☑ 254 
Q 246+9   T 255  ☑ 255 
Q 94+506  T 600  ☑ 600 
Q 360+77  T 437  ☑ 437 
Q 605+594 T 1199 ☑ 1199

--------------------------------------------------
Iteration 143
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 25s 554us/step - loss: 3.4861e-04 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9995
Q 1+607   T 608  ☑ 608 
Q 5+80    T 85   ☑ 85  
Q 622+85  T 707  ☑ 707 
Q 266+49  T 315  ☑ 315 
Q 43+527  T 570  ☑ 570 
Q 21+305  T 326  ☑ 326 
Q 542+472 T 1014 ☑ 1014
Q 73+469  T 542  ☑ 542 
Q 21+370  T 391  ☑ 391 
Q 541+632 T 1173 ☑ 1173

--------------------------------------------------
Iteration 156
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 550us/step - loss: 2.8675e-04 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9996
Q 80+231  T 311  ☑ 311 
Q 57+653  T 710  ☑ 710 
Q 834+126 T 960  ☑ 960 
Q 383+63  T 446  ☑ 446 
Q 46+535  T 581  ☑ 581 
Q 918+182 T 1100 ☑ 1100
Q 303+49  T 352  ☑ 352 
Q 657+149 T 806  ☑ 806 
Q 59+68   T 127  ☑ 127 
Q 29+562  T 591  ☑ 591 

--------------------------------------------------
Iteration 157
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 25s 554us/step - loss: 1.8116e-04 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 0.9996
Q 86+972  T 1058 ☑ 1058
Q 120+8   T 128  ☑ 128 
Q 941+3   T 944  ☑ 944 
Q 62+904  T 966  ☑ 966 
Q 6+947   T 953  ☑ 953 
Q 43+909  T 952  ☑ 952 
Q 713+350 T 1063 ☑ 1063
Q 306+306 T 612  ☑ 612 
Q 840+52  T 892  ☑ 892 
Q 443+9   T 452  ☑ 452 

--------------------------------------------------
Iteration 170
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 551us/step - loss: 1.6108e-04 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 0.9996
Q 97+66   T 163  ☑ 163 
Q 808+373 T 1181 ☑ 1181
Q 870+643 T 1513 ☑ 1513
Q 30+234  T 264  ☑ 264 
Q 2+577   T 579  ☑ 579 
Q 524+825 T 1349 ☑ 1349
Q 852+662 T 1514 ☑ 1514
Q 654+774 T 1428 ☑ 1428
Q 0+944   T 944  ☑ 944 
Q 198+50  T 248  ☑ 248 

--------------------------------------------------
Iteration 171
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 25s 548us/step - loss: 1.1928e-04 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 0.9996
Q 76+819  T 895  ☑ 895 
Q 830+865 T 1695 ☑ 1695
Q 502+16  T 518  ☑ 518 
Q 61+60   T 121  ☑ 121 
Q 590+795 T 1385 ☑ 1385
Q 611+866 T 1477 ☑ 1477
Q 942+32  T 974  ☑ 974 
Q 12+162  T 174  ☑ 174 
Q 6+137   T 143  ☑ 143 
Q 724+67  T 791  ☑ 791 

--------------------------------------------------
Iteration 184
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 556us/step - loss: 1.0793e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 0.9997
Q 1+373   T 374  ☑ 374 
Q 351+57  T 408  ☑ 408 
Q 193+8   T 201  ☑ 201 
Q 181+6   T 187  ☑ 187 
Q 9+809   T 818  ☑ 818 
Q 595+28  T 623  ☑ 623 
Q 23+213  T 236  ☑ 236 
Q 720+445 T 1165 ☑ 1165
Q 226+5   T 231  ☑ 231 
Q 62+969  T 1031 ☑ 1031

--------------------------------------------------
Iteration 185
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 25s 552us/step - loss: 0.0033 - acc: 0.9992 - val_loss: 0.0023 - val_acc: 0.9993
Q 9+674   T 683  ☑ 683 
Q 70+78   T 148  ☑ 148 
Q 981+634 T 1615 ☑ 1615
Q 479+149 T 628  ☑ 628 
Q 313+3   T 316  ☑ 316 
Q 785+83  T 868  ☑ 868 
Q 630+156 T 786  ☑ 786 
Q 396+217 T 613  ☑ 613 
Q 18+276  T 294  ☑ 294 
Q 618+35  T 653  ☑ 653 

--------------------------------------------------
Iteration 198
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 548us/step - loss: 4.9345e-04 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 0.9994
Q 584+1   T 585  ☑ 585 
Q 896+871 T 1767 ☑ 1767
Q 875+34  T 909  ☑ 909 
Q 2+272   T 274  ☑ 274 
Q 14+908  T 922  ☑ 922 
Q 81+393  T 474  ☑ 474 
Q 16+901  T 917  ☑ 917 
Q 99+428  T 527  ☑ 527 
Q 436+198 T 634  ☑ 634 
Q 762+1   T 763  ☑ 763 

--------------------------------------------------
Iteration 199
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/4500

In [20]:
model.save('rnn_model.h5')